In [ ]:
!pip install datasets > /dev/null


In [ ]:
!pip install torch torchvision pytorch-lightning > /dev/null

In [ ]:
!pip install mlflow
!databricks configure --host https://community.cloud.databricks.com/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
Username: mathieu.latournerie@epita.fr
Password: 
Repeat for confirmation: 
Error: The two entered values do not match.
Password: 
Repeat for confirmation: 


In [ ]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/mathieu.latournerie@epita.fr/Human action recognition")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2713494469571469', creation_time=1705593321771, experiment_id='2713494469571469', last_update_time=1705834446652, lifecycle_stage='active', name='/Users/mathieu.latournerie@epita.fr/Human action recognition', tags={'mlflow.experiment.sourceName': '/Users/mathieu.latournerie@epita.fr/Human '
                                 'action recognition',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'mathieu.latournerie@epita.fr',
 'mlflow.ownerId': '6684753197342109'}>

In [ ]:
# Load dataset
from datasets import load_dataset
ds = load_dataset("Bingsu/Human_Action_Recognition")
from torchvision.transforms import v2
import torch
from tqdm import tqdm

# Instantiate data module and model
transform = v2.Compose([
            v2.ToTensor(),
            v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
            v2.Resize(size=(224, 224), antialias=True),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
train_ds = [(transform(el['image']), el['labels']) for el in tqdm(ds['train'])]
#test_ds = [(transform(el['image']), el['labels']) for el in tqdm(ds['test'])]
del ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(
100%|██████████| 12600/12600 [00:37<00:00, 337.27it/s]


In [ ]:
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import models
import pytorch_lightning as pl
import numpy as np
mlflow.autolog()

class HumanActionDataModule(pl.LightningDataModule):
    def __init__(self, train_ds_in, val_split=0.2):
        super().__init__()
        self.train_ds_in = train_ds_in
        self.val_split = val_split

    def setup(self, stage=None):
        # Transform and split dataset
        val_size = int(len(self.train_ds_in) * self.val_split)
        train_size = len(train_ds) - val_size
        self.train_ds, self.val_ds = random_split(self.train_ds_in, [train_size, val_size])

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=16, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=64, num_workers=4)


class ResNetModel(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.resnet50(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.loss = nn.CrossEntropyLoss()
        self.val_losses = []
        self.val_accs = []
        self.epoch = 0
        self.transform = v2.Compose([
            v2.ToTensor(),
            v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
            v2.Resize(size=(224, 224), antialias=True),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def forward(self, x):
        return self.model(x)

    def predict_step(self, batch, batch_idx):
        x = self.transform(batch)
        out = self(x)
        return torch.argmax(out, dim=1)


    def training_step(self, batch, batch_idx):
        images, labels = batch[0], batch[1]
        outputs = self(images)
        loss = self.loss(outputs, labels)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch[0], batch[1]
        outputs = self(images)
        loss = self.loss(outputs, labels)

        labels_hat = torch.argmax(outputs, dim=1)
        val_acc = torch.sum(labels == labels_hat).item() / (len(labels) * 1.0)
        self.val_accs.append(val_acc)
        self.val_losses.append(loss.item())


    def on_validation_epoch_end(self):
        acc = np.mean(self.val_accs)
        print(f"Val acc:{acc}")
        loss = np.mean(self.val_losses)
        self.log(f'val_acc_{self.epoch}', acc)
        self.log(f'val_loss_{self.epoch}', loss)
        self.epoch += 1

    def test_step(self, batch, batch_idx):
      x, y = batch

      # implement your own
      out = self(x)
      loss = self.loss(out, y)


      # calculate acc
      labels_hat = torch.argmax(out, dim=1)
      test_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)

      # log the outputs!

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer



data_module = HumanActionDataModule(train_ds)



2024/01/21 13:22:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.
2024/01/21 13:22:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/01/21 13:22:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for pytorch_lightning.
2024/01/21 13:22:33 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


In [ ]:
with mlflow.start_run(run_name='accuracy test'):
  model = ResNetModel(num_classes=15, )
  model
  # Train the model
  trainer = pl.Trainer(max_epochs=10)
  trainer.fit(model, data_module)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, ple

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Val acc:0.0390625


Training: |          | 0/? [00:00<?, ?it/s]

...

Val acc:0.6637891344383058


Validation: |          | 0/? [00:00<?, ?it/s]

Val acc:0.6648398631840796


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
2024/01/21 13:30:06 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.9.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/01/21 13:30:06 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/01/21 13:30:20 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.4.23+cuda12.cudnn89) contains a local version label (+cuda12.cud

Uploading artifacts:   0%|          | 0/6 [00:00<?, ?it/s]